In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance

# json.dumps() 사용하기 위해 필요
import json


# Tool Implement
def get_ticker(inputs):  # Parameters object를 받음
    ddg = DuckDuckGoSearchAPIWrapper(backend="api")
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


# 손익계산서
def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)

    # LLM은 String을 읽기 때문에 String 형태로 변환해야함
    # stock은 데이터프레임(스프레드시트) 형태로 들어옴
    # .to_json(): 데이터프레임 형태를 json 형태로 변환
    # json.dumps(): json 형태를 다시 문자열로 변환
    return json.dumps(stock.income_stmt.to_json())  # stmt: statement


# 대차대조표
def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


# 100일 동안의 주식 퍼포먼스
def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    # history에 들어가보면 바꿀 수 있는 parameter를 확인할 수 있음
    return json.dumps(stock.history(period="3mo").to_json())


# Tools에 넘겨줄 Description
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [9]:
from openai import OpenAI

client = OpenAI()

# # OpenAI에 assistant를 생성함
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4o-mini",
#     # tools가 어떻게 작동될 지에 대한 description을 List형태로 넘겨줘야 함
#     tools=functions,
# )

# print(assistant)
# # assistant.id를 확인할 수 있는데 이것은 앞으로도 계속 사용됨
# # assistant를 생성한 후에 OpenAI의 Playground를 이용해 assistant를 테스트해 볼 수 있음

assistant_id = "asst_sgTX3TAy06FPU13MqF1zOeqw"

In [10]:
# thread 생성
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread
# Assistant의 thread에 가보면 생성된 것을 확인할 수 있음

Thread(id='thread_7JKdEGF3LKWFHWZUNQGl8t9U', created_at=1737789588, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [11]:
thread_id = "thread_7JKdEGF3LKWFHWZUNQGl8t9U"

In [ ]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

In [ ]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

In [ ]:
def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

In [ ]:
get_run(run_id, thread_id).status

In [ ]:
get_messages(thread.id)